In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
import time
import pandas as pd


In [ ]:
# Cria semente para numeros randomicos
rnd = np.random
rnd.seed(0)

In [ ]:
#Parametro que define a quantidade de clientes
quantidade_clientes =  7 # numero de clientes

In [ ]:
# Define o ponto dos clientes com base na quantidade - O deposito será acrescentado posteriormente, e será convencionado como ponto 1
Clientes = [i for i in range(2, quantidade_clientes + 2)]
Clientes

In [ ]:
# Define todos os pontos do modelo, acrescentado o deposito como primeiro ponto
Pontos = [1] + Clientes
Pontos

In [ ]:
# Traça todos os arcos possiveis para representar os caminhos possiveis entre dois pontos distintos
Caminhos = [(i,j) for i in Pontos for j in Pontos if i!=j]
print('Quantidade: ' + str(len(Caminhos)))
Caminhos

In [ ]:
# Definindo os veículos do modelo
Veiculos = [1,2]
Veiculos

In [ ]:
# Os Arcos associados com os custos. Para esta implementação estamos abstraindo que a unica variavel envolvida é a distancia
custo ={(i, j): { 'c':rnd.randint(quantidade_clientes)+i+j } for i, j in Caminhos}
print('Quantidade: ' + str(len(custo)))
custo

In [ ]:
# Inicio da medição do tempo de CPU utilizado
inicio  = time.process_time()

In [ ]:
# Definindo todas a rotas possiveis
# retorna todas a permutacoes de rotas possiveis partindo do primeiro elemento que é o deposito
# Acrescentamos o deposito no final para ajudar no calculo do retorno (Ex. pegar o valor de (4,1))
# Neste momento já montamos as rotas atendendo a premissa que a mesma rota não passa no mesmo cliente duas vezes
# Também evitamos a criação de subrotas
def todas_rotas(seq, quantidade):
  #permuta todas a opcoes com base na quantidade de destinos desejados. Fixamos o primeiro item como deposito de chegada e saida
  return [[seq[0]] + list(rest) + [seq[0]] for rest in itertools.permutations(seq[1:],quantidade)]
contador = 1
Rotas = []
#repito as permutacoes aumentando a quantidade de itens envolvidos
while (contador < len(Pontos)):
  Rotas = Rotas + todas_rotas(Pontos,contador) 
  contador   = contador + 1
print('Quantidade: ' + str(len(Rotas)))
Rotas

In [ ]:
# Calculando o custo de cada rota
# Recuperamos cada rota. De cada rota recuperamos o custo de cada trecho, somamos os trechos
# de forma a encontrar o custo total da rota
# o resultado vai para um dicionario que tem a rota e a somatoria dos trechos.

def custo_do_trecho(i, j):

    try:
        custo.get((i, j)).get('c')
    except AttributeError:
        result = 0
    else:
        result = custo.get((i, j)).get('c')
    return result


def custo_da_rota(rota):
    resultado = 0

    for i in range(0,len(rota)-1):
        resultado = resultado + custo_do_trecho(rota[i], rota[i + 1])
    return resultado

lista_custos={}
i=0
for rota in Rotas:
    curso_rota = custo_da_rota(rota)
    lista_custos[i]=(rota,curso_rota)
    i = i +1 
print('Quantidade: ' + str(len(lista_custos)))
 
lista_custos

In [ ]:
# Filtragem da premissa que um cliente não pode ser atendido mais de uma vez
def verificaSeClienteAtendidoApenasUmaVez(rota_para_restricao):
  inclui_rota = True
  lista_rotas_temp = rota_para_restricao[0][0]+rota_para_restricao[1][0]
  for cliente in Clientes:
    if(lista_rotas_temp.count(cliente)>1):
        inclui_rota = False
        break;
  return inclui_rota


In [ ]:
# Filtragem da premissa que todos os clientes devem ser atendidos
def verificaSeTodosOsPontosAtendidos(rota_para_restricao):
  inclui_rota = True
  lista_rotas_temp = rota_para_restricao[0][0]+rota_para_restricao[1][0]
  for cliente in Clientes:
    if(lista_rotas_temp.count(cliente)==0):
        inclui_rota = False
        break;
  return inclui_rota

In [ ]:
# Cruza todas as rotas possiveis para os dois veiculos
# Convencionamos que a primeira rota para veiculo 1 e a segunda para o veiculo 2
# aplicando verificas para futuramente calcular apenas as rotas que nao desobedecem 
# alguma das premissas: Cliente atendido apenas uma vez e todos os clientes atendidos
lista_rotas_combinada={}
i=0
for rota_combinada in itertools.combinations(lista_custos.values(),2):
  if(verificaSeClienteAtendidoApenasUmaVez(rota_combinada) and verificaSeTodosOsPontosAtendidos(rota_combinada)):
    lista_rotas_combinada[i] = rota_combinada
  i = i+1
print('Quantidade: ' + str(len(lista_rotas_combinada)))
lista_rotas_combinada

In [ ]:
#Calcula o custo total da combinação das duas rotas
# é somado o custo de cada rota e armazenado o valor total
lista_rotas_combinada_custo={}
i=0
for rota_para_calculo in lista_rotas_combinada.values():
  lista_rotas_combinada_custo[i]= (rota_para_calculo,rota_para_calculo[0][1]+rota_para_calculo[1][1])
  i = i+1
print('Quantidade: ' + str(len(lista_rotas_combinada_custo)))
lista_rotas_combinada_custo

In [ ]:
rotas_resultantes = pd.DataFrame.from_dict(lista_rotas_combinada_custo.values())
  
rotas_resultantes 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
rotas_resultantes_csv  = rotas_resultantes.to_csv('/content/drive/MyDrive/DADOS_VRP/DATASET/VRP_TESTE3.csv',sep=';',index_label='index',header=['trechos','custo'])
rotas_resultantes_csv 